# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama


# İş Problemi

##### Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları
##### oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete
##### ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.
##### Örneğin: Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

### PROJE GÖREVLERİ

#### GÖREV 1: Aşağıdaki soruları yanıtlayınız.

In [2]:
import pandas as pd

#pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
df = pd.read_excel('gezinomidata/miuul_gezinomi.xlsx')

In [3]:
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low


In [5]:
# Soru 2: Kaç unique şehir vardır? Frekansları nedir?
print(df["SaleCityName"].nunique())
print("-------------------------------------------")
print(df["SaleCityName"].value_counts())

6
-------------------------------------------
SaleCityName
Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: count, dtype: int64


In [7]:
# Soru 3: Kaç unique Concept vardır?
print(df["ConceptName"].nunique())
print("--------------------------")
# Soru 4: Hangi Concept'dan kaçar tane satış gerçekleşmiş?k
df["ConceptName"].value_counts()


3
--------------------------


ConceptName
Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: count, dtype: int64

In [8]:
# Soru 5: Şehirlere göre satışlardan toplam ne kadar kazanılmış?
df.groupby("SaleCityName").agg({"Price": "sum"})

,Price
SaleCityName,
Antalya,2.041911e+06
Aydın,5.732960e+05
Diğer,1.545723e+05
Girne,2.706503e+04
Muğla,6.658422e+05
İzmir,1.659348e+05


In [9]:
# Soru 6: Concept türlerine göre göre ne kadar kazanılmış?
df.groupby("ConceptName").agg({"Price": "sum"})

,Price
ConceptName,
Herşey Dahil,3.332911e+06
Oda + Kahvaltı,1.213084e+05
Yarım Pansiyon,1.744024e+05


In [12]:
#result = df.groupby("ConceptName").agg({"Price": "sum"})
#result["Price"] = result["Price"].apply(lambda x: "{:,.2f}".format(x))
#print(result)

In [10]:
# Soru 7: Şehirlere göre PRICE ortalamaları nedir?
df.groupby(by=['SaleCityName']).agg({"Price": "mean"})

,Price
SaleCityName,
Antalya,64.521474
Aydın,53.855896
Diğer,47.707497
Girne,59.483579
Muğla,62.461746
İzmir,66.267905


In [11]:
# Soru 8: Conceptlere  göre PRICE ortalamaları nedir?
df.groupby(by=['ConceptName']).agg({"Price": "mean"})

,Price
ConceptName,
Herşey Dahil,62.672260
Oda + Kahvaltı,50.252010
Yarım Pansiyon,49.030742


In [13]:
# Soru 9: Şehir-Concept kırılımında PRICE ortalamaları nedir?
df.groupby(by=["SaleCityName", 'ConceptName']).agg({"Price": "mean"})

Price
SaleCityName ConceptName              
Antalya      Herşey Dahil    64.519028
             Oda + Kahvaltı  63.504816
             Yarım Pansiyon  67.190613
Aydın        Herşey Dahil    53.995467
             Oda + Kahvaltı  34.458787
             Yarım Pansiyon  30.016543
Diğer        Herşey Dahil    84.771093
             Oda + Kahvaltı  37.599082
             Yarım Pansiyon  42.113440
Girne        Herşey Dahil    97.680970
             Oda + Kahvaltı  39.775536
             Yarım Pansiyon  53.247838
Muğla        Herşey Dahil    63.020053
             Oda + Kahvaltı  59.037763
             Yarım Pansiyon  45.120924
İzmir        Herşey Dahil    74.701519
             Oda + Kahvaltı  41.320342
             Yarım Pansiyon  59.610053

#### GÖREV 2: satis_checkin_day_diff değişkenini EB_Score adında yeni bir kategorik değişkene çeviriniz.

In [16]:
bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]

In [20]:
df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels=labels)

In [21]:
df.head(50).to_excel("eb_scorew.xlsx", index=False)

#### GÖREV 3: Şehir,Concept, [EB_Score,Sezon,CInday] kırılımında ücret ortalamalarına ve frekanslarına bakınız


In [27]:
# Şehir-Concept-EB Score kırılımında ücret ortalamaları
print(
    df.groupby(by=["SaleCityName", 'ConceptName', "EB_Score" ])
    .agg({"Price": ["mean", "count"]})
)  # Parantez kapatılmamış
print("*****************************************************************************************")
# Şehir-Concept-Sezon kırılımında ücret ortalamaları
print(
    df.groupby(by=["SaleCityName", "ConceptName", "Seasons"])
    .agg({"Price": ["mean", "count"]})
)  # Parantez kapatılmamış
print("*****************************************************************************************")

# Şehir-Concept-CInday kırılımında ücret ortalamaları
print(
    df.groupby(by=["SaleCityName", "ConceptName", "CInDay"])
    .agg({"Price": ["mean", "count"]})
)  # Parantez kapatılmamış


                                                    Price       
                                                     mean  count
SaleCityName ConceptName    EB_Score                            
Antalya      Herşey Dahil   Last Minuters       62.752916  14148
                            Potential Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early Bookers       66.494885   3281
             Oda + Kahvaltı Last Minuters       65.353740    503
...                                                   ...    ...
İzmir        Oda + Kahvaltı Early Bookers       66.928111      3
             Yarım Pansiyon Last Minuters       57.788126    636
                            Potential Planners  64.804043    198
                            Planners            60.507887     47
                            Early Bookers       65.905583     14

[72 rows x 2 columns]
*******************************************************************

#### GÖREV 4: City-Concept-Season kırılımın çıktısını PRICE'a göre sıralayınız.

###### Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
###### Çıktıyı agg_df olarak kaydediniz.


In [29]:
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}).sort_values("Price", ascending=False)
agg_df.head(20)

Price
SaleCityName ConceptName    Seasons            
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
İzmir        Yarım Pansiyon High      87.657303
Diğer        Herşey Dahil   Low       87.310882
                            High      83.787273
İzmir        Herşey Dahil   High      74.748363
                            Low       74.308287
Antalya      Yarım Pansiyon High      73.259029
             Oda + Kahvaltı High      66.267969
             Herşey Dahil   High      64.920065
Muğla        Herşey Dahil   High      63.384036
Antalya      Yarım Pansiyon Low       62.001678
             Herşey Dahil   Low       61.552427
Muğla        Oda + Kahvaltı High      61.264691
Antalya      Oda + Kahvaltı Low       60.666730
Girne        Yarım Pansiyon High      58.383668
İzmir        Yarım Pansiyon Low       55.350626
Aydın        Herşey Dahil   High      54.946560
İzmir        Oda + Kahvaltı High      54.141797
Girne        Yarım Pansiyon Low       48.578903

#### GÖREV 5: Indekste yer alan isimleri değişken ismine çeviriniz.

In [33]:
##### Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
##### Bu isimleri değişken isimlerine çeviriniz.
##### İpucu: reset_index()
agg_df.reset_index(inplace=True)
agg_df.head()


,level_0,index,SaleCityName,ConceptName,Seasons,Price
0,0,0,Girne,Herşey Dahil,High,103.935448
1,1,1,Girne,Herşey Dahil,Low,90.935944
2,2,2,İzmir,Yarım Pansiyon,High,87.657303
3,3,3,Diğer,Herşey Dahil,Low,87.310882
4,4,4,Diğer,Herşey Dahil,High,83.787273


#### GÖREV 6: Yeni level based satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.


In [35]:
# sales_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
agg_df['sales_level_based'] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: '_'.join(x).upper(), axis=1)

agg_df.head(3)

,level_0,index,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,0,0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,1,1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,2,2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH


#### GÖREV 7: Personaları segmentlere ayırınız.

In [36]:
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz
# segmentleri betimleyiniz
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})

Price                        
              mean         max         sum
SEGMENT                                   
D        33.366288   39.479630  300.296592
C        44.887654   54.141797  403.988887
B        60.274498   64.920065  542.470478
A        82.467833  103.935448  742.210498

#### GÖREV 8: Oluşan son df'i price değişkenine göre sıralayınız.


In [37]:
# "ANTALYA_HERŞEY DAHIL_HIGH" hangi segmenttedir ve ne kadar ücret beklenmektedir?
agg_df.sort_values(by="Price")

,level_0,index,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
35,35,35,Aydın,Yarım Pansiyon,Low,25.271799,AYDIN_YARIM PANSIYON_LOW,D
34,34,34,Aydın,Oda + Kahvaltı,High,30.387530,AYDIN_ODA + KAHVALTI_HIGH,D
33,33,33,Aydın,Yarım Pansiyon,High,32.626152,AYDIN_YARIM PANSIYON_HIGH,D
32,32,32,Muğla,Yarım Pansiyon,Low,32.681812,MUĞLA_YARIM PANSIYON_LOW,D
31,31,31,İzmir,Oda + Kahvaltı,Low,33.555235,İZMIR_ODA + KAHVALTI_LOW,D
30,30,30,Aydın,Herşey Dahil,Low,33.680699,AYDIN_HERŞEY DAHIL_LOW,D
29,29,29,Diğer,Oda + Kahvaltı,High,34.839543,DIĞER_ODA + KAHVALTI_HIGH,D
28,28,28,Muğla,Oda + Kahvaltı,Low,37.774192,MUĞLA_ODA + KAHVALTI_LOW,D
27,27,27,Girne,Oda + Kahvaltı,High,39.479630,GIRNE_ODA + KAHVALTI_HIGH,D
26,26,26,Diğer,Yarım Pansiyon,High,39.731916,DIĞER_YARIM PANSIYON_HIGH,C


In [38]:
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]

,level_0,index,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
9,9,9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH,B
